In [ ]:
import cv2
import numpy as np
from czifile import CziFile

# --- Load CZI image ---
def load_czi_as_numpy(path):
    with CziFile(path) as czi:
        img = czi.asarray()
    return np.squeeze(img)  # Remove singleton dims

# --- Global variables ---
zoom = 1.0
ix, iy = -1, -1
drawing = False
cropped = None

# --- Mouse callback to draw rectangle and extract ROI ---
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, cropped, zoomed_image

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = zoomed_image.copy()
            cv2.rectangle(img_copy, (ix, iy), (x, y), (0, 255, 0), 2)
            cv2.imshow('Image Viewer', img_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        x0, y0 = min(ix, x), min(iy, y)
        x1, y1 = max(ix, x), max(iy, y)
        cropped = zoomed_image[y0:y1, x0:x1]
        cv2.imshow('Cropped Region', cropped)

# --- Resize image by zoom factor ---
def resize_image(img, scale):
    h, w = img.shape[:2]
    return cv2.resize(img, (int(w * scale), int(h * scale)))

# --- Main Viewer ---
def view_image(img):
    global zoom, zoomed_image

    cv2.namedWindow('Image Viewer')
    cv2.setMouseCallback('Image Viewer', draw_rectangle)

    while True:
        zoomed_image = resize_image(img, zoom)
        cv2.imshow('Image Viewer', zoomed_image)

        key = cv2.waitKey(0)

        if key == ord('+') or key == ord('='):  # Zoom in
            zoom *= 1.2
        elif key == ord('-') or key == ord('_'):  # Zoom out
            zoom = max(0.2, zoom / 1.2)
        elif key == ord('s') and cropped is not None:
            cv2.imwrite("extracted_region.png", cropped)
            print("Extracted region saved as 'extracted_region.png'")
        elif key == 27:  # ESC key
            break

    cv2.destroyAllWindows()

# --- Run ---
if __name__ == '__main__':
    czi_path = 'your_image.czi'  # Replace with actual path
    image = load_czi_as_numpy(czi_path)

    # If multi-channel (e.g., RGB), use 3 channels
    if image.ndim > 2 and image.shape[-1] in (3, 4):
        pass
    else:
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    view_image(image)
